PREPARATION FOR MULTIPLE SEQUENCE ALIGNMENT (MSA)

In [ ]:
#code to count how many transcripts there are within a given read folder
import pandas as pd
import pyfaidx as f
import os
import glob 
os.getcwd()

glob.glob("../fasta/*") 

myfiles = glob.glob("../fasta/*") 

outfile = open("NumberOfTranscriptsPerFile.txt", "w")
for file in myfiles:
    count = 0 
    lines = open(file).readlines()
    for line in lines:
        if line.startswith(">"):
            count = count + 1
    outfile.write(f"{file}\t{count}\n") 
outfile.close()

In [ ]:
#code to tabulate the number of transcripts per sample and present data in a histogram.

import pandas as pd
tab = pd.read_csv("C:\\Users\\lukew\\Desktop\\LabProjectBiocomp\\Notebook\\NumberOfTranscriptsPerFile.txt", sep = "\t", names=['file', 'count'])

import matplotlib.pyplot as plt  
%matplotlib inline   

plt.hist(tab['count'], color = 'red', bins = 20)  
plt.xlabel("number of transcripts per file")
plt.ylabel("frequency")
plt.title("Number of Transcripts per File")

RUNNING MULTIPLE SEQUENCE ALIGNMENT (MSA)

In [ ]:
#MaRNAV1 Segment 1 MSA 1st attempt

command = "mafft --adjustdirection /home/lj1893/LabProjectBiocomp/fasta/VirusSequenceAssembly/RDRPPositiveContigs/RDRPPositiveContigs.fasta > /home/lj1893/LabProjectBiocomp/fasta/VirusSequenceAssembly/RDRPPositiveContigs/MultipleSequenceAlignment.fasta"
os.system(command)

In [ ]:
#refining the dataset to exclude the spurious study

PositiveNinetyFive = tab[tab["percent_identity"] > 95]

gb = PositiveNinetyFive.groupby('file') 
RDRPPositiveContigs = open("/home/lj1893/LabProjectBiocomp/fasta/VirusSequenceAssembly/RDRPPositiveContigs/RDRPPositiveContigsFiltered.fasta", "w")
gb.groups 


for group in gb.groups:  
    one = group.split("/")[-1]
    two = one.split(".out")[0]
    if "SRR20668688.SRR20668689.SRR20668690.SRR20668691" not in two:
        fasta = f"/home/lj1893/LabProjectBiocomp/fasta/assembled_fastas/{two}.fasta"
        FastaFileContents = f.Fasta(fasta)
        small_table = gb.get_group(group) 
        for SequenceName in small_table['query']:
            sequence = FastaFileContents[SequenceName][:]
            s = sequence.seq
            RDRPPositiveContigs.write(f">{two}_{SequenceName}\n{s}\n")
            print(len(s))
RDRPPositiveContigs.close()

In [ ]:
#MaRNAV1 Segment 1 MSA 2st attempt
command = "mafft --adjustdirection /home/lj1893/LabProjectBiocomp/fasta/VirusSequenceAssembly/RDRPPositiveContigs/RDRPPositiveContigsFiltered.fasta > /home/lj1893/LabProjectBiocomp/fasta/VirusSequenceAssembly/RDRPPositiveContigs/MultipleSequenceAlignmentFiltered.fasta"
os.system(command)

In [ ]:
#MaRNAV1 Segment 1 MSA 3rd attempt, this time using CIAlign
%%bash
CIAlign --infile /home/lj1893/LabProjectBiocomp/fasta/VirusSequenceAssembly/RDRPPositiveContigs/MultipleSequenceAlignmentFiltered.fasta \
--outfile_stem /home/lj1893/LabProjectBiocomp/fasta/VirusSequenceAssembly/RDRPPositiveContigs/Seg1Alignment_ \
--deduplicate_ids \
--crop_divergent --crop_divergent_buffer 10 --crop_divergent_min_prop_nongap 0.1 --remove_insertions --insertion_min_size 1 --plot_output

In [ ]:
#MaRNAV1 Segment 2 MSA 3rd using CIAlign
%%bash
CIAlign --infile /home/lj1893/LabProjectBiocomp/fasta/VirusSequenceAssembly/HypotheticalProteinPositiveContigs/MultipleSequenceAlignmentFiltered.fasta \
--outfile_stem /home/lj1893/LabProjectBiocomp/fasta/VirusSequenceAssembly/HypotheticalProteinPositiveContigs/Seg2Alignment_ \
--deduplicate_ids \
--crop_divergent --crop_divergent_buffer 10 --crop_divergent_min_prop_nongap 0.1 --remove_insertions --insertion_min_size 1 --insertion_max_size 600 --plot_output 

DETERMINING THE 5' AND 3' UTRs OF SEGMENT 1

In [ ]:
#Segment1 5' end
#processing and printing the 0-61

import pyfaidx
fasta = pyfaidx.Fasta("/home/lj1893/LabProjectBiocomp/fasta/VirusSequenceAssembly/RDRPPositiveContigs/Seg1Alignment__cleaned.fasta")

starts = set()

for record in fasta:
    name = record.name #turning the name of the file into a variable name
    seq = str(record) #converting record to a string gives you back the sequence, stored as variable seq
    seq_start = seq[0:60] #seq_start is storing the first 61 values of the string as a variable
    seq_start_nogaps = seq_start.replace("-", "")

    if len(seq_start_nogaps) != 0:
        starts.add(seq_start_nogaps)

print(starts)

In [ ]:
#Segment1 5' end
# sorting the set of ends by discarding any shorter sequences that are parts of larger sequences. Keep the longer sequences.

starts.sort(key=len, reverse=True)
result_starts = []

for s in starts:
    if not any(s in other for other in result_starts):
        result_starts.append(s)

In [ ]:
#Segment1 3' end
#processing and printing the 3086-end
import pyfaidx
fasta = pyfaidx.Fasta("/home/lj1893/LabProjectBiocomp/fasta/VirusSequenceAssembly/RDRPPositiveContigs/Seg1Alignment__cleaned.fasta")

ends = set()

for record in fasta:
    name = record.name #turning the name of the file into a variable name
    seq = str(record) #converting record to a string gives you back the sequence, stored as variable seq
    seq_ends = seq[3086:] #seq_start is storing the first 61 values of the string as a variable
    seq_ends_nogaps = seq_ends.replace("-", "")

    if len(seq_ends_nogaps) != 0:
        ends.add(seq_ends_nogaps)

print(ends)

In [ ]:
#Segment1 3' end
# sorting the set of ends by discarding any shorter sequences that are parts of larger sequences. Keep the longer sequences.

ends.sort(key=len, reverse=True)
result_ends = []

for e in ends:
    if not any(e in other for other in result_ends):
        result_ends.append(e)

In [ ]:
#Cropping alignment to keep columns 61 to 261
#Removing gap only columns
#Building 5' consensus buffer

%%bash
CIAlign --infile /home/lj1893/LabProjectBiocomp/fasta/VirusSequenceAssembly/RDRPPositiveContigs/Seg1Alignment__cleaned.fasta --outfile_stem /home/lj1893/LabProjectBiocomp/fasta/VirusSequenceAssembly/RDRPPositiveContigs/ConsensusBufferLeft_seg1 --get_section --section_start 61 --section_end 261 --make_consensus --consensus_type majority_nongap

In [ ]:
#Cropping alignment to keep columns 3085 to end
#Removing gap only columns
#Building 3' consensus buffer
%%bash
CIAlign --infile /home/lj1893/LabProjectBiocomp/fasta/VirusSequenceAssembly/RDRPPositiveContigs/Seg1Alignment__cleaned.fasta --outfile_stem /home/lj1893/LabProjectBiocomp/fasta/VirusSequenceAssembly/RDRPPositiveContigs/ConsensusBufferRight_seg1 --get_section --section_start 2885 --section_end 3085 --make_consensus --consensus_type majority_nongap


In [ ]:
#combining 5' ends + 5' consensus buffer and transfering them to a fasta file

left_fasta = pyfaidx.Fasta("/home/lj1893//LabProjectBiocomp/fasta/VirusSequenceAssembly/RDRPPositiveContigs/ConsensusBufferLeft_seg1_with_consensus.fasta")
right_fasta = pyfaidx.Fasta("/home/lj1893//LabProjectBiocomp/fasta/VirusSequenceAssembly/RDRPPositiveContigs/ConsensusBufferRight_seg1_with_consensus.fasta")

left_consensus_buffer_fasta = str(left_fasta['consensus'][:]) 
right_consensus_buffer_fasta = str(right_fasta['consensus'][:]) 

i = 1
for start in result_starts:   
    seq = start + left_consensus_buffer_fasta
    out = open(f"/home/lj1893/LabProjectBiocomp/fasta/VirusSequenceAssembly/RDRPPositiveContigs/StartsPlusBufferLeft/StartL_{i}.fasta", 'w')
    out.write(f">start_{i}\n{seq}\n")
    out.close() 
    i = i + 1

In [ ]:
#combining 3' ends + 3' consensus buffer and transfering them to a fasta file

left_fasta = pyfaidx.Fasta("/home/lj1893//LabProjectBiocomp/fasta/VirusSequenceAssembly/RDRPPositiveContigs/ConsensusBufferLeft_seg1_with_consensus.fasta")
right_fasta = pyfaidx.Fasta("/home/lj1893//LabProjectBiocomp/fasta/VirusSequenceAssembly/RDRPPositiveContigs/ConsensusBufferRight_seg1_with_consensus.fasta")

left_consensus_buffer_fasta = str(left_fasta['consensus'][:])
right_consensus_buffer_fasta = str(right_fasta['consensus'][:]) 
i = 1
for end in result_ends:   
    seq = right_consensus_buffer_fasta + end
    out = open(f"/home/lj1893/LabProjectBiocomp/fasta/VirusSequenceAssembly/RDRPPositiveContigs/EndsPlusBufferRight/EndR_{i}.fasta", 'w')
    out.write(f">end_{i}\n{seq}\n")
    out.close() 
    i = i + 1

DETERMINING THE 5' AND 3' UTRs OF SEGMENT 2

In [ ]:
#Segment2 5' end
#processing and printing the 0-39

import pyfaidx
fasta = pyfaidx.Fasta("/home/lj1893/LabProjectBiocomp/fasta/VirusSequenceAssembly/HypotheticalProteinPositiveContigs/Seg2Alignment__cleaned.fasta")

starts = set()

for record in fasta:
    name = record.name 
    seq = str(record)
    seq_start = seq[0:39] 
    seq_start_nogaps = seq_start.replace("-", "")

    if len(seq_start_nogaps) != 0:
        starts.add(seq_start_nogaps)

print(starts)

In [ ]:
#Segment2 5' end
# sorting the set of ends by discarding any shorter sequences that are parts of larger sequences. Keep the longer sequences.

starts.sort(key=len, reverse=True)
result_starts = []

for s in starts:
    if not any(s in other for other in result_starts):
        result_starts.append(s)

In [ ]:
#Cropping alignment to keep columns 39 to 239
#Removing gap only columns
#Building 5' consensus buffer
%%bash
CIAlign --infile /home/lj1893/LabProjectBiocomp/fasta/VirusSequenceAssembly/HypotheticalProteinPositiveContigs/Seg2Alignment__cleaned.fasta --outfile_stem /home/lj1893/LabProjectBiocomp/fasta/VirusSequenceAssembly/HypotheticalProteinPositiveContigs/ConsensusBufferLeft_seg2 --get_section --section_start 39 --section_end 239 --make_consensus --consensus_type majority_nongap

In [ ]:
#combining 5' ends + 5' consensus buffer and transfering them to a fasta file

left_fasta = pyfaidx.Fasta("/home/lj1893/LabProjectBiocomp/fasta/VirusSequenceAssembly/HypotheticalProteinPositiveContigs/ConsensusBufferLeft_seg2_with_consensus.fasta")

left_consensus_buffer_fasta = str(left_fasta['consensus'][:]) #taking the bottom line of the left fasta (which is the consensus sequnece - see one note for pic)

i = 1
for start in result_starts:    #adding the consensus buffer sequnece to the ends (left)
    seq = start + left_consensus_buffer_fasta
    out = open(f"/home/lj1893/LabProjectBiocomp/fasta/VirusSequenceAssembly/HypotheticalProteinPositiveContigs/StartsPlusBufferLeft/StartL_{i}.fasta", 'w')
    out.write(f">start_{i}\n{seq}\n")
    out.close() 
    i = i + 1

In [ ]:
REMAPPING OF ORIGINAL READS TO THESE VARIABLE END FRAGMENTS (CODE WRITTEN BY MY SUPERVISOR KATHERINE BROWN)

In [ ]:
#find all start and end sequences (made up of combined variable regions and 200 nucleotide consensus buffer)
#build a bowtie2 index for each start & end fasta file - allows the software to map reads to this file

import pandas as pd
import numpy as np
import pyfaidx
import glob
import os

starts = sorted(glob.glob("start_seqs/*fasta"))
ends = sorted(glob.glob("end_seqs/*fasta"))

for st in starts:
    stem = st.replace(".fasta", "")
    statement = f"bowtie2-build {st} {stem}"
    os.system(statement)

for en in ends:
    stem = en.replace(".fasta", "")
    statement = f"bowtie2-build {en} {stem}"
    os.system(statement)

reads = glob.glob("reads/*fastq*")

In [ ]:
# code to perform the remapping of the reads to the starts and the ends
#code can be re-used for Segment 2 by changing out the file names


# Loop through the files containing the reads
for rfile in reads:
    # Get the sample ID from the file name
    rnam = rfile.split("/")[-1].split("_final")[0]
    # Loop through each start sequence
    for st in starts:
        # Find the index of the start sequence
        stem = st.replace(".fasta", "")
        ind = stem.split("_")[-1]
        
        # Run bowtie to map the reads to the start sequence
        # ignore-quals is because lots of the FASTQ files have no 
        # quality scores
        # Convert the bowtie2 output file to a more compressed format
        # and filter out poor quality and unmapped reads
        statement = f"""bowtie2 -x {stem} -U {rfile} --ignore-quals |\
                       samtools view -F4 -q 40 -b |\
                       samtools sort > bams_new/{rnam}_{ind}_start.bam"""
        os.system(statement)
        # Index the output file so the data can be accessed more quickly
        statement = f"samtools index bams_new/{rnam}_{ind}_start.bam"
        os.system(statement)
    for en in ends:
        # Find the index of the end sequence
        stem = en.replace(".fasta", "")
        ind = stem.split("_")[-1]
        # Run bowtie to map the reads to the end sequence
        # ignore-quals is because lots of the FASTQ files have no 
        # quality scores
        # Convert the bowtie2 output file to a more compressed format
        # and filter out poor quality and unmapped reads
        statement = f"""bowtie2 -x {stem} -U {rfile} --ignore-quals |\
                       samtools view -F4 -q 40 -b |\
                       samtools sort > bams_new/{rnam}_{ind}_end.bam"""
        os.system(statement)
        # Index the output file so the data can be accessed more quickly
        statement = f"samtools index bams_new/{rnam}_{ind}_end.bam"
        os.system(statement)


In [ ]:

starts = sorted(glob.glob("start_seqs_seg2/*fasta"))

for st in starts:
    stem = st.replace(".fasta", "")
    statement = f"bowtie2-build {st} {stem}"
    os.system(statement)

reads = ['reads/SRR20668524.SRR20668525.SRR20668526.SRR20668527_final.fastq.gz',
         'reads/SRR20668844.SRR20668845.SRR20668846.SRR20668847_final.fastq.gz']

for rfile in reads:
    rnam = rfile.split("/")[-1].split("_final")[0]
    for st in starts:
        stem = st.replace(".fasta", "")
        ind = stem.split("_")[-1]
        if not os.path.exists(f"bams_new_s2/{rnam}_{ind}_start.bam"):
            statement = f"""bowtie2 -x {stem} -U {rfile} --ignore-quals |\
                           samtools view -F4 -q 40 -b |\
                           samtools sort > bams_new_s2/{rnam}_{ind}_start.bam"""
            os.system(statement)
            statement = f"samtools index bams_new_s2/{rnam}_{ind}_start.bam"
            os.system(statement)


CONSENSUS SEQUENCES OF THE VARIABLE ENDS AND CENTRAL REGIONS WERE MANUALLY COMBINED USING A TEXT EDITOR TO GIVE THE OVERALL CONSENSUS SEQUENCE FOR EACH MARNAV1 SEGMENT.